In [11]:
# set parameters
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import dmg5e
import estats5e
import tfb

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

In [2]:
# import data
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','XP','AC','HP Mean','AB','DPR Mean'
dfD['eHP Mean']  = dfD.apply(lambda row: estats5e.effHP(row['HP Mean'], row['AC']), axis=1)
dfD['eDPR Mean'] = dfD.apply(lambda row: estats5e.effDPR(row['DPR Mean'], row['AB']), axis=1)
dfD['eXP Mean']  = dfD.apply(lambda row: estats5e.effXP(row['HP Mean'], row['AC'], row['DPR Mean'], row['AB']), axis=1)
dfD.set_index('CR', inplace=True)

xp2cr  = np.poly1d(np.polyfit(dfD['eXP Mean'],  dfD.index, 3))
hp2cr  = np.poly1d(np.polyfit(dfD['eHP Mean'],  dfD.index, 3))
dpr2cr = np.poly1d(np.polyfit(dfD['eDPR Mean'], dfD.index, 3))

df0 = pd.read_csv('../../assets/data/monsters.csv')
df0 = df0[df0['adj DPR'].gt(0)]
df0['eHP']  = df0.apply(lambda row: estats5e.effHP(row['adj HP'], row['adj AC']), axis=1)
df0['eDPR'] = df0.apply(lambda row: estats5e.effDPR(row['adj DPR'], row['adj AB']), axis=1)
df0['eXP']  = df0.apply(lambda row: estats5e.effXP(row['adj HP'], row['adj AC'], row['adj DPR'], row['adj AB']), axis=1)

df0['D-eCR']  = df0.apply(lambda row: float(np.interp(row['eHP'], dfD['eHP Mean'],  dfD.index, left=0, right=30)), axis=1)
df0['O-eCR']  = df0.apply(lambda row: float(np.interp(row['eDPR'], dfD['eDPR Mean'],  dfD.index, left=0, right=30)), axis=1)
df0['eCR']  = df0.apply(lambda row: float(np.interp(row['eXP'], dfD['eXP Mean'],  dfD.index, left=0, right=30)), axis=1)

df0['eHP Ratio'] = df0.apply(lambda row: row['eHP']/dfD.loc[row['CR'], 'eHP Mean'], axis=1)
df0['eDPR Ratio'] = df0.apply(lambda row: row['eDPR']/dfD.loc[row['CR'], 'eDPR Mean'], axis=1)
df0['eCR Delta'] = df0['eCR'] - df0['CR']
df0['eXP Ratio'] = df0['eXP'] / df0['XP']
df0['eXP Skew'] = (df0['eDPR Ratio'] - df0['eHP Ratio'])/(df0['eDPR Ratio'] + df0['eHP Ratio'])

df0['D-CR DMG'] = df0.apply(lambda row: dmg5e.monster_defensive_challenge_rating(row['adj HP'], row['adj AC']), axis=1)
df0['O-CR DMG'] = df0.apply(lambda row: dmg5e.monster_offensive_challenge_rating(row['adj DPR'], row['adj AB']), axis=1)
df0['C-CR DMG'] = df0.apply(lambda row: np.floor(dmg5e.monster_challenge_rating(row['adj HP'], row['adj AC'], row['adj DPR'], row['adj AB'])), axis=1)
df0['C-CR DMG Delta'] = df0['C-CR DMG'] - df0['CR']
df0['XP DMG'] = df0.apply(lambda row: dfD.loc[row['C-CR DMG'], 'XP'], axis=1)
df0['XP DMG Ratio'] = df0['XP DMG'] / df0['XP']
#df0['C-CR DMG'] = 0.5*(df0['D-CR DMG'] + df0['O-CR DMG'])
df0['CR spread'] = df0['CR'] + np.random.rand(len(df0.index.tolist())) - 0.5

len(df0.index.tolist())
print('Includes {} monsters in database'.format(len(df0.index.to_list())))

Includes 2143 monsters in database


In [ ]:
# Fig. 1: calculated XP vs target XP
xCol = 'XP'
yCol = 'eXP'

conf = 0.8
df1 = df0[df0['CR'].between(0,30)]
dfG = df1[[xCol,yCol]].groupby(xCol).mean().reset_index()
dfU = df1[[xCol,yCol]].groupby(xCol).quantile(0.5 + conf/2).reset_index()
dfL = df1[[xCol,yCol]].groupby(xCol).quantile(0.5 - conf/2).reset_index()

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='XP',
            type='log',
        ),
        yaxis=dict(
            title_text='calculated XP',
            type='log',
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster confidence interval and average
fig.add_trace(go.Scatter(x=dfG[xCol], y=dfU[yCol], line_width=0, legendgroup='mean', showlegend=False, hoverinfo='skip'))
fig.add_trace(go.Scatter(x=dfG[xCol], y=dfL[yCol], line_width=0, name=f'{conf:.0%} confidence', legendgroup='mean', hoverinfo='skip', fill='tonexty', fillcolor='rgba(0, 0, 255, 0.4)'))
fig.add_trace(go.Scatter(
    x=dfG[xCol], y=dfG[yCol],
    mode='lines', 
    name='mean',
    legendgroup='mean', 
    line=dict(color='blue'),
    hovertemplate='<b>mean</b><br>' + 'XP %{x:,.0f}<br>' + 'Calc XP %{y:,.0f}<br>' + '<extra></extra>'
))

# plot reference line
fig.add_trace(go.Scatter(
    x=dfG[xCol], y=dfG[xCol],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hovertemplate='<b>DMG</b><br>' + 'XP %{x:,.0f}' + '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-calc-xp-vs-xp-log-log-large', selector={'name': 'none'})
    tfb.save_fig_html(fig, format='small', name=f'./fig-calc-xp-vs-xp-log-log-small', selector={'name': 'none'})

In [5]:
# Fig. 2: calculated XP vs target XP
xCol = 'CR'
yCol = 'eCR'

conf = 0.6
#df1 = df0[df0['CR'].between(0,30)]
dfG = df1[[xCol,yCol]].groupby(xCol).mean().reset_index()
dfU = df1[[xCol,yCol]].groupby(xCol).quantile(0.5 + conf/2).reset_index()
dfL = df1[[xCol,yCol]].groupby(xCol).quantile(0.5 - conf/2).reset_index()

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,30],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated CR',
            range=[0,30],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster confidence interval and average
fig.add_trace(go.Scatter(x=dfG[xCol], y=dfU[yCol], line_width=0, legendgroup='mean', showlegend=False, hoverinfo='skip'))
fig.add_trace(go.Scatter(x=dfG[xCol], y=dfL[yCol], line_width=0, name=f'{conf:.0%} confidence', legendgroup='mean', hoverinfo='skip', fill='tonexty', fillcolor='rgba(0, 0, 255, 0.4)'))
fig.add_trace(go.Scatter(
    x=dfG[xCol], y=dfG[yCol],
    mode='lines', 
    name='mean',
    legendgroup='mean', 
    line=dict(color='blue'),
    hovertemplate='<b>mean</b><br>' + 'CR %{x:,.0f}<br>' + 'Calc CR %{y:,.1f}<br>' + '<extra></extra>'
))

# plot reference line
fig.add_trace(go.Scatter(
    x=dfG[xCol], y=dfG[xCol],
    mode='lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hovertemplate='<b>DMG</b><br>' + 'CR %{x:,.0f}' + '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-xp-cr-vs-cr-large', selector={'name': 'none'})
    tfb.save_fig_html(fig, format='small', name=f'./fig-xp-cr-vs-cr-small', selector={'name': 'none'})

In [ ]:
# Fig. 3: DMG CR deltas
xCol = 'CR'
yCols = ['eCR Delta','C-CR DMG Delta']
aCols = [xCol] + yCols
colors = ['rgba(0,0,250,1)','rgba(250,0,0,1)']
fillcolors = ['rgba(0,0,250,0.3)','rgba(250,0,0,0.3)']

conf = 0.6
df1 = df0[df0[xCol].between(0,30) & df0['Category'].isin(['generic','legendary','unique'])]
dfM = df1[aCols].groupby(xCol).mean().reset_index()
dfU = df1[aCols].groupby(xCol).quantile(0.5 + conf/2).reset_index()
dfL = df1[aCols].groupby(xCol).quantile(0.5 - conf/2).reset_index()

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,30],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='calculated CR delta',
            range=[-6,6],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)

for yCol, color, fillcolor in zip(yCols, colors, fillcolors):
    fig.add_trace(go.Scatter(
        x=dfM[xCol], y=dfM[yCol],
        mode='lines', 
        name=f'{yCol} - mean',
        line=dict(color=color),
        hovertemplate='<b>mean</b><br>' + 'CR %{x:,.0f}<br>' + 'CR delta %{y:,.1f}<br>' + '<extra></extra>'
    ))

    fig.add_trace(go.Scatter(x=dfG[xCol], y=dfU[yCol], line_width=0, legendgroup=f'{yCol} confidence', showlegend=False, hoverinfo='skip'))
    fig.add_trace(go.Scatter(x=dfG[xCol], y=dfL[yCol], line_width=0, name=f'{yCol} - {conf:.0%} confidence', legendgroup=f'{yCol} confidence', hoverinfo='skip', fill='tonexty', fillcolor=fillcolor))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-ecr-dmg-cr-delta-vs-cr-large', selector={'name': 'none'})
    tfb.save_fig_html(fig, format='small', name=f'./fig-ecr-dmg-cr-delta-vs-cr-small', selector={'name': 'none'})

In [ ]:
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,30],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='XP ratio',
            range=[0,3],
            tick0=0, dtick=0.5,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster data
df1 = df0[df0['CR'].between(0,30) & df0['Category'].isin(['generic','legacy','legendary','unique'])]
fig.add_trace(go.Scatter(
    x=df1['CR spread'], y=df1['eXP Ratio'],
    mode='markers', 
    name='monsters',
    marker=dict(color='blue', opacity=1),
    customdata = np.stack((df1['Monster'], df1['CR'], df1['eXP Ratio']), axis=-1),
    hovertemplate = '<b>%{customdata[0]}</b><br>'
        + 'CR: %{customdata[1]}<br>'
        + 'eXP ratio: %{customdata[2]:.2f}'
        + '<extra></extra>'
))

# plot average
xCol = 'CR'
yCol = 'eXP Ratio'


dfM = df1[[xCol,yCol]].groupby(xCol).mean().reset_index()
fig.add_trace(go.Scatter(
    x=dfM[xCol], y=dfM[yCol],
    mode='lines', 
    name='mean',
    line=dict(color='blue'),
    hovertemplate='<b>mean</b><br>' + 'CR %{x:,.0f}<br>' + 'XP ratio %{y:,.2f}<br>' + '<extra></extra>'
))

conf = 0.6
dfU = df1[[xCol,yCol]].groupby(xCol).quantile(0.5 + conf/2).reset_index()
dfL = df1[[xCol,yCol]].groupby(xCol).quantile(0.5 - conf/2).reset_index()
fig.add_trace(go.Scatter(x=dfG[xCol], y=dfU[yCol], line_width=0, legendgroup='confidence', showlegend=False, hoverinfo='skip'))
fig.add_trace(go.Scatter(x=dfG[xCol], y=dfL[yCol], line_width=0, name=f'{conf:.0%} confidence', legendgroup='confidence', hoverinfo='skip', fill='tonexty', fillcolor='rgba(0, 0, 255, 0.2)'))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-xp-ratio-vs-cr-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-xp-ratio-vs-cr-small')
    pass

In [9]:
# eDPR vs eHP
CRs = [6,9,12,15]
cats = ['generic','legendary','unique']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
colors = colors[0:len(CRs)]

mask = df0['CR'].isin(CRs) & df0['Category'].isin(cats)
xMin = np.min(df0.loc[mask, 'eDPR'])
xMax = np.max(df0.loc[mask, 'eDPR'])
yMin = np.min(df0.loc[mask, 'eHP'])
yMax = np.max(df0.loc[mask, 'eHP'])

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        width=600,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='effective damage per round',
            type='log',
            #range=np.log10([0.95*xMin,1.05*xMax]), 
            #tick0=1, dtick=0.1,
            #minor=dict(tick0=0, dtick=0.1),
        ),
        yaxis=dict(
            title_text='effective hit points',
            type='log',
            #range=np.log10([0.95*yMin,1.05*yMax]), 
            #tick0=0, dtick=1,
            #minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


for cr, c in zip(CRs, colors):
    df1 = df0[df0['CR'].isin([cr]) & df0['Category'].isin(cats)]
    fig.add_trace(go.Scatter(
        x=df1['eDPR'], y=df1['eHP'],
        mode='markers', 
        name=f'CR = {cr:.0f}',
        legendgroup=f'CR = {cr:.0f}',
        marker_color=c,
        hovertemplate=f'<b>CR {cr:.0f}</b><br>' + 'eDPR %{x:,.0f}<br>' + 'eHP %{y:,.0f}<br>' + '<extra></extra>',
    ))

x = np.linspace(0.9*xMin, 1.1*xMax, 50, endpoint=True)
XP = dfD['XP'].values
for cr, c in zip(CRs, colors):
    fig.add_trace(go.Scatter(
        x=x, y=4*dfD.loc[cr]['eXP Mean']/x,
        mode='lines', 
        line=dict(color=c, dash='dash'),
        showlegend=False,
        name=f'CR = {cr:.0f}',
        legendgroup=f'CR = {cr:.0f}',
        hoverinfo='skip',
    ))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-ehp-vs-edpr-log-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-ehp-vs-edpr-log-small')
    pass

In [8]:
# eHP Ratio

yCol = 'eHP Ratio'

conf = 0.6
df1 = df0[df0['CR'].between(1,30)]
dfM = df1[['CR',yCol]].groupby('CR').median()
dfU = df1[['CR',yCol]].groupby('CR').quantile(0.5 + conf/2)
dfL = df1[['CR',yCol]].groupby('CR').quantile(0.5 - conf/2)

# CR
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,30],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text=yCol,
            range=[0,2],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)


# add reference line
fig.add_trace(go.Scatter(
    x=[1,30], y=[1,1],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    name='DMG',
    hoverinfo='skip',
))

# add average value
fig.add_trace(go.Scatter(
    x=dfM.index, y=dfM[yCol],
    mode='lines', 
    line=dict(color='rgba(0,0,250,1)', dash='solid'),
    name=f'{yCol} - mean',
    hovertemplate='<b>mean</b><br>' + 'CR %{x:,.0f}<br>' + yCol + ' %{y:,.1f}<br>' + '<extra></extra>'

))

# add confidence interval
fig.add_trace(go.Scatter(x=dfM.index, y=dfU[yCol], line_width=0, legendgroup=f'{yCol} confidence', showlegend=False, hoverinfo='skip'))
fig.add_trace(go.Scatter(x=dfM.index, y=dfL[yCol], line_width=0, name=f'{yCol} - {conf:.0%} confidence', legendgroup=f'{yCol} confidence', hoverinfo='skip', fill='tonexty', fillcolor='rgba(0,0,250,0.2)'))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-ehp-ratio-vs-cr-large', selector={'name': 'none'})
    #tfb.save_fig_html(fig, format='small', name=f'./fig-ehp-ratio-vs-cr-small', selector={'name': 'none'})
    pass

In [7]:
# eHP Ratio

yCol = 'eDPR Ratio'

conf = 0.6
df1 = df0[df0['CR'].between(1,30)]
dfM = df1[['CR',yCol]].groupby('CR').median()
dfU = df1[['CR',yCol]].groupby('CR').quantile(0.5 + conf/2)
dfL = df1[['CR',yCol]].groupby('CR').quantile(0.5 - conf/2)

# CR
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='CR',
            range=[0,30],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text=yCol,
            range=[0,2],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
            bgcolor='rgba(0,0,0,0)',
        )
    )
)


# add reference line
fig.add_trace(go.Scatter(
    x=[1,30], y=[1,1],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    name='DMG',
    hoverinfo='skip',
))

# add average value
fig.add_trace(go.Scatter(
    x=dfM.index, y=dfM[yCol],
    mode='lines', 
    line=dict(color='rgba(0,0,250,1)', dash='solid'),
    name=f'{yCol} - mean',
    hovertemplate='<b>mean</b><br>' + 'CR %{x:,.0f}<br>' + yCol + ' %{y:,.1f}<br>' + '<extra></extra>'

))

# add confidence interval
fig.add_trace(go.Scatter(x=dfM.index, y=dfU[yCol], line_width=0, legendgroup=f'{yCol} confidence', showlegend=False, hoverinfo='skip'))
fig.add_trace(go.Scatter(x=dfM.index, y=dfL[yCol], line_width=0, name=f'{yCol} - {conf:.0%} confidence', legendgroup=f'{yCol} confidence', hoverinfo='skip', fill='tonexty', fillcolor='rgba(0,0,250,0.2)'))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    #fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-edpr-ratio-vs-cr-large', selector={'name': 'none'})
    #tfb.save_fig_html(fig, format='small', name=f'./fig-edpr-ratio-vs-cr-small', selector={'name': 'none'})
    pass